###Connect Drive

In [18]:
# path_root = '/content/drive/MyDrive/datas/'
path_root = '/Users/sanghunoh/Develops/repository/own/study_machinglearning/datas/'
# 작업 경로 설정
import os
os.chdir(path_root)
!pwd

/Users/sanghunoh/Develops/repository/own/study_machinglearning/datas


In [19]:
# !unzip -o ./mpie_30_shuffle.zip -d ./mpie_30_shuffle

In [20]:
!ls

HW3Data_1.mat                           digitimages_binary.zip
HW4_COIL20.mat                          iris.zip
HW5Data_1.mat                           mpie_30_shuffle
HW5Data_2.mat                           mpie_30_shuffle.zip
README.md                               results_MLP_label_0_202205300952.pickle
dataCh4_7.zip                           results_MLP_label_3_202205300952.pickle


###Load Dataset

In [21]:
path = path_root + 'mpie_30_shuffle/'

import pandas as pd

def loadDatasetFromCSV(_datafilename, _labelfilename, label_column):
  _feature_csv = pd.read_csv(path + _datafilename, dtype=np.float32, header=None) # image features of train data

  _label_csv = pd.read_csv(path + _labelfilename, dtype=np.float32, header=None) # labels of train data
  _label = _label_csv[label_column] # get label you want

  return _feature_csv, _label

In [22]:
import numpy as np

In [23]:
def preprocessDataset(label_column):
  # train
  train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', label_column)
  # test
  validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', label_column)

  return train_feature_reshape, train_label, validation_feature_reshape, validation_label

In [24]:
train_feature, train_label, validation_feature, validation_label = preprocessDataset(1)
type(train_feature), train_feature.shape, type(train_label), train_label.shape, validation_feature.shape, validation_label.shape

(pandas.core.frame.DataFrame,
 (18777, 1024),
 pandas.core.series.Series,
 (18777,),
 (5086, 1024),
 (5086,))

####check model

In [25]:
import tensorflow as tf
from tensorflow.keras import layers

class_cnt = len(np.unique(train_label))
# class_cnt

model = tf.keras.Sequential()
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(class_cnt, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

_epochs = 10
model.fit(train_feature, train_label)

2022-05-30 10:19:57.650310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


587/587 [==============================] - 4s 7ms/step - loss: 1.1503 - accuracy: 0.5933


In [26]:
# input_layer = layers.Input(shape=(1024,))  # input: 1024 nodes
# x = tf.keras.layers.Flatten(name='Flatten')(input_layer)
# x = layers.Activation('relu')(layers.Dense(256, input_dim=1024)(x))  # hidden: 256 nodes
# x = layers.BatchNormalization()(x)
# y = layers.Activation('softmax')(layers.Dense(class_cnt)(x))    # number of output nodes : class_no (it changes depend on label you selected)
# model = tf.keras.Model(x, y)
# model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# model.fit(train_feature, train_label)

###MPL CNN with BatchNormalization

In [27]:
import numpy as np

class_cnt = len(np.unique(train_label))
class_cnt

6

In [28]:
import tensorflow as tf
from tensorflow.keras import layers

def make_model(_class_cnt, _hidden_layers=1, dense_count=256, _kernel_initializer='glorot_uniform', _activation='relu', _optimizer='SGD', batch_normal=False, epochs=2, label_class=0):
  _model = tf.keras.Sequential()
  _model.add(layers.Dense(dense_count, input_dim=1024, kernel_initializer=_kernel_initializer, activation=_activation)) 
  if batch_normal:
    _model.add(layers.BatchNormalization())
  for idx in range(_hidden_layers):
    _model.add(layers.Dense(dense_count, kernel_initializer=_kernel_initializer, activation=_activation)) 
    if batch_normal:
      _model.add(layers.BatchNormalization())
  _model.add(layers.Dense(_class_cnt, activation='softmax')) 

  _model.compile(loss='sparse_categorical_crossentropy', optimizer=_optimizer, metrics=['accuracy'])

  return _model
model = make_model(_class_cnt=6)    
# tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, expand_nested=True, show_layer_activations=True) #layer_range=?, 

###callback function for fit time

In [29]:
import time

class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
TimeHistory()

### fit model

In [30]:
rows = train_feature
# _batch_size = int(len(list(rows)) / (len(list(rows))/2))   # Just Check Model params quickly
_batch_size = len(list(rows))

def model_fit(_param, _dataset):
  tf.keras.backend.clear_session()

  (_train_feature, _train_label, _validation_feature, _validation_label) = _dataset
  _param['_class_cnt'] = len(np.unique(_train_label))
  print(_param['epochs'], _param['_class_cnt'])

  model = make_model(**_param)
  time_callback = TimeHistory()
  history = model.fit(_train_feature, _train_label, epochs=_param['epochs'], validation_data=(_validation_feature, _validation_label), callbacks=[time_callback], steps_per_epoch=_batch_size)
  # history = model.fit(_train_feature, _train_label, epochs=_epochs, callbacks=[time_callback])
  execution_time = sum(time_callback.times)
  return model, history, execution_time, _param
dataset = preprocessDataset(1)  
param = {'_hidden_layers':1, 'dense_count':256, '_kernel_initializer':'glorot_uniform', '_activation':'relu', '_optimizer':'Adam', 'batch_normal':False, 'epochs':2}
model_fit(param, dataset)  

2 6
Epoch 1/2
   5/1024 [..............................] - ETA: 13s - loss: 1.6758 - accuracy: 0.5053 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0059s vs `on_train_batch_end` time: 0.0115s). Check your callbacks.


2022-05-30 10:20:03.164644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1014/1024 [============================>.] - ETA: 0s - loss: 1.1241 - accuracy: 0.6320

2022-05-30 10:20:08.218731: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1024/1024 [==============================] - 6s 5ms/step - loss: 1.1225 - accuracy: 0.6323 - val_loss: 1.0109 - val_accuracy: 0.6516
Epoch 2/2
1024/1024 [==============================] - 5s 5ms/step - loss: 0.9586 - accuracy: 0.6713 - val_loss: 0.9794 - val_accuracy: 0.6422


(<keras.engine.sequential.Sequential at 0x1768c1460>,
 10.829020261764526,
 {'_hidden_layers': 1,
  'dense_count': 256,
  '_kernel_initializer': 'glorot_uniform',
  '_activation': 'relu',
  '_optimizer': 'Adam',
  'batch_normal': False,
  'epochs': 2,
  '_class_cnt': 6})

### fit with multi params

In [31]:
# params = [
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           ]

In [7]:
# param 6
params = list()
# param_key_list = list(param.keys())
# _hidden_layers_list = [0, 5]
# dense_count_list = [64, 256]
# _kernel_initializer_list = [None, 'glorot_uniform']
# _activation_list = ['sigmoid', 'relu']
# _optimizer_list= ['SGD', 'Adam']
# batch_normal_list = [False, True]
# epochs_list = [10, 40]
_hidden_layers_list = [3]
dense_count_list = [8, 16]
_kernel_initializer_list = ['glorot_uniform']
_activation_list = ['sigmoid', 'relu']
_optimizer_list= ['SGD', 'Adam']
batch_normal_list = [False, True]
epochs_list = [5, 10]

for _hidden_layers in _hidden_layers_list:
  for dense_count in dense_count_list:
    for _kernel_initializer in _kernel_initializer_list:
      for _activation in _activation_list:
        for _optimizer in _optimizer_list:
          for batch_normal in batch_normal_list:
            for epochs in epochs_list:
              param_dict = dict()
              param_dict['_hidden_layers'] = _hidden_layers
              param_dict['dense_count'] = dense_count
              param_dict['_kernel_initializer'] = _kernel_initializer
              param_dict['_activation'] = _activation
              param_dict['_optimizer'] = _optimizer
              param_dict['batch_normal'] = batch_normal
              param_dict['epochs'] = epochs
              # print(param_dict)
              params.append(param_dict)
len(params), 
for param in params[::4]:
  print(param)

{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'Adam', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'relu', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'relu', '_optimizer': 'Adam', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers': 3, 'dense_count': 16, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers': 3, 'dense_count': 16, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'Adam', 'batch_normal': False, 'epochs': 5}
{'_hidden_layers'

In [33]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y%m%d%H%M")
print("date and time =", dt_string)

import pickle

now = 2022-05-30 10:20:13.998830
date and time = 202205301020


In [34]:
# params = params[6:11]
label_classes = [0, 3, 5]

results_list = list()
for label_class in label_classes:
  dataset = preprocessDataset(label_class)
  results = list()          
  for idx, param in enumerate(params):
    param['label_class'] = label_class
    print('-'*5 +'[ '+ str(idx) + ' ]'+ '-'*5 + str(param))
    results.append(model_fit(param, dataset))
  # save
  with open(f'results_MLP_label_{label_class}_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)
  results_list = results_list + results


-----[ 0 ]-----{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 5, 'label_class': 0}
5 30
Epoch 1/5
   6/1024 [..............................] - ETA: 10s - loss: 3.4829 - accuracy: 0.0175 

2022-05-30 10:20:15.302445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1016/1024 [============================>.] - ETA: 0s - loss: 3.4098 - accuracy: 0.0334

2022-05-30 10:20:20.257353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1024/1024 [==============================] - 6s 5ms/step - loss: 3.4097 - accuracy: 0.0335 - val_loss: 3.3929 - val_accuracy: 0.0385
Epoch 2/5
1024/1024 [==============================] - 5s 5ms/step - loss: 3.3865 - accuracy: 0.0393 - val_loss: 3.3857 - val_accuracy: 0.0407
Epoch 3/5
1024/1024 [==============================] - 5s 5ms/step - loss: 3.3825 - accuracy: 0.0402 - val_loss: 3.3846 - val_accuracy: 0.0409
Epoch 4/5
1024/1024 [==============================] - 5s 5ms/step - loss: 3.3813 - accuracy: 0.0406 - val_loss: 3.3843 - val_accuracy: 0.0409
Epoch 5/5
1024/1024 [==============================] - 4s 4ms/step - loss: 3.3820 - accuracy: 0.0390 - val_loss: 3.3842 - val_accuracy: 0.0405
-----[ 1 ]-----{'_hidden_layers': 3, 'dense_count': 8, '_kernel_initializer': 'glorot_uniform', '_activation': 'sigmoid', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 10, 'label_class': 0}
10 30
Epoch 1/10
   9/1024 [..............................] - ETA: 6s - loss: 3.4734 - accuracy: 

...

1024/1024 [==============================] - 13s 12ms/step - loss: 0.6337 - accuracy: 0.7503 - val_loss: 0.4811 - val_accuracy: 0.8134
Epoch 2/5
1024/1024 [==============================] - 11s 11ms/step - loss: 0.3357 - accuracy: 0.8796 - val_loss: 0.1834 - val_accuracy: 0.9434
Epoch 3/5
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2855 - accuracy: 0.8993 - val_loss: 0.2701 - val_accuracy: 0.8991
Epoch 4/5
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2886 - accuracy: 0.8951 - val_loss: 0.2316 - val_accuracy: 0.9202
Epoch 5/5
1024/1024 [==============================] - 10s 9ms/step - loss: 0.2439 - accuracy: 0.9121 - val_loss: 0.2854 - val_accuracy: 0.8958
-----[ 31 ]-----{'_hidden_layers': 3, 'dense_count': 16, '_kernel_initializer': 'glorot_uniform', '_activation': 'relu', '_optimizer': 'Adam', 'batch_normal': True, 'epochs': 10, 'label_class': 5, '_class_cnt': 2}
10 4
Epoch 1/10


2022-05-30 11:51:37.931391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1023/1024 [============================>.] - ETA: 0s - loss: 0.6381 - accuracy: 0.7522

2022-05-30 11:51:50.242608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1024/1024 [==============================] - 13s 12ms/step - loss: 0.6379 - accuracy: 0.7523 - val_loss: 0.3938 - val_accuracy: 0.8539
Epoch 2/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.3310 - accuracy: 0.8799 - val_loss: 0.5248 - val_accuracy: 0.8124
Epoch 3/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2638 - accuracy: 0.9071 - val_loss: 0.3408 - val_accuracy: 0.8757
Epoch 4/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2525 - accuracy: 0.9101 - val_loss: 0.2688 - val_accuracy: 0.8976
Epoch 5/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2225 - accuracy: 0.9222 - val_loss: 0.2566 - val_accuracy: 0.9029
Epoch 6/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2095 - accuracy: 0.9251 - val_loss: 0.1912 - val_accuracy: 0.9304
Epoch 7/10
1024/1024 [==============================] - 11s 11ms/step - loss: 0.2113 - accuracy: 0.9249 - val_loss: 0.5207 - val_accura

In [35]:

# save all
with open(f'results_MLP_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results_list, f, pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ram://7f75eb2f-3196-466c-a0f1-9d575b5a58c2/assets
INFO:tensorflow:Assets written to: ram://2e010c6e-36f5-4931-b276-2c80fb83444b/assets
INFO:tensorflow:Assets written to: ram://af5043a6-273b-450c-bfe5-c9a1b18af329/assets
INFO:tensorflow:Assets written to: ram://86f833c3-eb42-4b77-b28a-f7ac6710caf6/assets
INFO:tensorflow:Assets written to: ram://f411e6ce-5f20-41e3-bafa-0722b8eb3e73/assets
INFO:tensorflow:Assets written to: ram://6c66bd65-51c6-4c67-85c2-f8fda692e321/assets
INFO:tensorflow:Assets written to: ram://d9141d17-fafa-46e7-8515-f43790a49ec1/assets
INFO:tensorflow:Assets written to: ram://a1222991-1028-4d85-9297-b1ed1a58224c/assets
INFO:tensorflow:Assets written to: ram://fa632e71-9dc0-420f-a44f-76e0a4420cbb/assets
INFO:tensorflow:Assets written to: ram://2cd333e8-8694-49d1-aeb1-90de2fe7b55f/assets
INFO:tensorflow:Assets written to: ram://1718fc52-94e4-4d31-9ea9-5ab2a9a31fee/assets
INFO:tensorflow:Assets written to: ram://d3969bf1-a7a1-4f7d-8cad-